In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [2]:
def get_classifier(b):
    strt=b.find('classifier:__choice__')+25
    classifier=""
    for i in range(strt,len(b)-strt):
        if b[i]!=",":
            classifier+=b[i]
        else:
            classifier=classifier[:-1]
            break
    return classifier

In [3]:
#metric="accuracy"
#metric="f1_score"
#metric="recall"
metric="precision"

In [4]:
docs=[pd.read_csv("logs/Results10.csv"),pd.read_csv("logs/Results30.csv"),pd.read_csv("logs/Results60.csv"),pd.read_csv("logs/Results4.csv")]

In [5]:
total=[]
names=["complete","ensemble","meta learning","vanilla"]
parts=["","e_","m_","v_"]
for doc in docs:
    classifiers=[]
    for index, row in doc.iterrows():
        scores=[row["sklearn_"+parts[0]+metric+"_mean"],row["sklearn_"+parts[1]+metric+"_mean"],row["sklearn_"+parts[2]+metric+"_mean"],row["sklearn_"+parts[3]+metric+"_mean"]]
        ind=(scores.index(max(scores)))
        try:
            best=[row['Unnamed: 0'],get_classifier(row["sklearn_"+parts[ind]+"model_1"]),row["sklearn_"+parts[ind]+metric+"_mean"]]
        except:
            best=[row['Unnamed: 0'],None,None]
        classifiers.append(best)
    total.append(classifiers)


In [6]:
merged=[]
for i in range(len(total[0])):
    a=[total[0][i][2],total[1][i][2],total[2][i][2],total[3][i][2]]
    b=[x for x in a if x!=None]
    if  b:
        ind=(b.index(max(b)))
        merged.append(total[ind][i])
    else:
        merged.append([total[0][i][0], None,None])
        pass

In [7]:
metas=pd.read_csv("Complete_Sheet.csv")

In [8]:
metas=metas.drop(columns=['num_imput_type'])

In [9]:
ex_ft=['nr_classes', 'nr_instances', 'log_nr_instances', 'nr_features',
       'log_nr_features', 'missing_val', 'nr_categorical_features',
       'dataset_ratio', 'labels_mean', 'labels_std', 'skew_min', 'skew_std',
       'skew_mean', 'skew_q1', 'skew_q3', 'skew_max', 'kurtosis_min',
       'kurtosis_std', 'kurtosis_mean', 'kurtosis_q1', 'kurtosis_q3',
       'kurtosis_max', 'rho_min', 'rho_max', 'rho_mean', 'rho_std',
       'class_entropy', 'prob_min', 'prob_mean', 'prob_std', 'prob_max',
       'norm_entropy_min', 'norm_entropy_mean', 'norm_entropy_std',
       'norm_entropy_max', 'mi_min', 'mi_mean', 'mi_std', 'mi_max',
       'equiv_nr_feat', 'noise_signal_ratio', 'dataset']

In [10]:
metas=metas[ex_ft]


In [11]:
merged_df = pd.DataFrame(merged,columns=["dataset","classifier","score"]) 

In [12]:
final=merged_df.merge(metas)

In [13]:
final=(final.dropna())

In [14]:
final.to_csv("merged_meta_"+metric+".csv",index=False)